# 1. Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
from pyomo.environ import *
import plotly.express as px
import plotly.graph_objects as go
from highspy import Highs
from pprint import pprint

# 2. Production Details for different products

In [ ]:
products_list = ['A', 'B', 'C'] # Monitor, TV, Refrigerator
time_periods = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

required_hours_per_unit = {'A': 2, 'B': 3, 'C': 5}

initial_inventory = {'A': 200, 'B': 200, 'C': 200}
space_factors = {'A': 0.2, 'B': 0.5, 'C': 1.0}

over_time_fraction = 0.25 # 25% of Regular hours are allowed for over time

scenes = ['low', 'med', 'high']
probs = {
    'low': 0.25,
    'med': 0.5,
    'high': 0.25
}

In [ ]:
def create_prod_data(prod, time_periods=time_periods):
    temp_pd = pd.DataFrame({
        't': time_periods,
        'base_demand': np.random.randint(100, 300, len(time_periods)),
        'unit_production_cost': np.random.randint(5, 15, len(time_periods)),  # Material costs
        'unit_holding_cost': np.random.randint(2, 6, len(time_periods)),      # Storage costs
        'unit_backorder_cost': np.random.randint(15, 40, len(time_periods)),  # Customer penalty
        'unit_reg_labor_cost': np.random.randint(15, 25, len(time_periods)),  # $/hour regular
        'unit_ot_labor_cost': np.random.randint(25, 40, len(time_periods)),   # $/hour OT (1.5-2x)
        'hiring_cost': np.random.randint(500, 1000, len(time_periods)),       # $$$ per worker
        'firing_cost': np.random.randint(300, 800, len(time_periods)),        # $$$ per worker
    })
    
    temp_pd['product'] = prod
    temp_pd['pt_index'] = temp_pd['product'].astype(str) + '_' + temp_pd['t'].astype(str)
        
    temp_pd['low_demand'] = np.random.poisson(temp_pd['base_demand'] * 0.8)
    temp_pd['med_demand'] = np.random.poisson(temp_pd['base_demand'])
    temp_pd['high_demand'] = np.random.poisson(temp_pd['base_demand'] * 1.2)
    temp_pd['prob_low'] = 0.25
    temp_pd['prob_med'] = 0.5
    temp_pd['prob_high'] = 0.25

    return temp_pd

In [ ]:
prod_pd = pd.DataFrame()
prod_pd = pd.concat(create_prod_data(p, time_periods) for p in products_list)
prod_pd = prod_pd.set_index('pt_index')
prod_pd.to_csv('product_data.csv')
prod_pd.head(12)